<a href="https://colab.research.google.com/github/jsilbergDS/UniTox/blob/main/Ablations/Upload_Processor_DICT_No_CoT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyPDF2
!pip install openai
!pip install tqdm
!pip install openai

In [ ]:
import requests
import io
import PyPDF2
from bs4 import BeautifulSoup
import pandas as pd

import json
import requests
import pandas as pd
from openai import OpenAI
import time
import numpy as np
from tqdm import tqdm
import re

client = OpenAI(api_key="")

In [ ]:
df_to_use = pd.read_csv("/content/drive/MyDrive/ClinicalTrials/dailymed_full_text_final.csv")
dictrank_df = pd.read_csv("Table S1-Table 1.csv")

In [ ]:
dictrank_df = pd.read_csv("Table S1-Table 1.csv")
dictrank_df["lower"] = dictrank_df["Generic/Proper Name(s)"].str.lower()
df_to_use["lower"] = df_to_use["generic_name"].str.lower()
dictrank_df = dictrank_df.merge(df_to_use,on="lower",how="inner")
dictrank_df = dictrank_df.groupby("lower").first().reset_index()

In [ ]:
dictrank_df

In [ ]:
#USE ME

drugs_list = []
drugs_merged_list = []
drugs_initial_responses = []
drugs_no_less_most_responses = []
drugs_yes_no_responses = []
drugs_descriptive_responses = []
answers_list = []
initial_prompts = []
no_less_most_prompts = []
yes_no_prompts = []
descriptive_prompts = []
moiety_list = []
fda_label_list = []
set_list = []

for index, row in tqdm(dictrank_df.iterrows(), total=dictrank_df.shape[0]):
    #url=url.replace("https://fdalabel.fda.gov:8443","https://nctr-crs.fda.gov")
    #if "nctr-crs.fda.gov" in url:
    #print(plain_text)
    #elif "accessdata" in url:
    #    r = requests.get(url)
    #    f = io.BytesIO(r.content)

    #    reader = PyPDF2.PdfReader(f)
    #    plain_text = "\n".join([reader.pages[i].extract_text().strip() for i in range(0,len(reader.pages))])
    #    plain_text=plain_text.strip()

    no_less_most_prompt = """Given the above information about a drug, answer 'was this drug associated with No Cardiotoxicity, Less Cardiotoxicity, or Most Cardiotoxicity?'
        Now answer with just one word: No, Less, or Most"""
    yes_no_prompt = """Given the above information about a drug, answer 'was this drug associated with Cardiotoxicity?'
        Now, answer with just one word: Yes or No"""

    no_less_most_response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content":row["full_text"]},
        {"role": "user", "content":no_less_most_prompt}
        ],
        stream=False, temperature=0

        )
    yes_no_response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content":row["full_text"]},
        {"role": "user", "content":yes_no_prompt}
        ],
        stream=False, temperature=0

        )
    print(no_less_most_response.choices[0].message.content.strip()," ",yes_no_response.choices[0].message.content.strip())
    drugs_list.append(row["generic_name"])
    moiety_list.append(row["moiety"])
    drugs_no_less_most_responses.append(no_less_most_response.choices[0].message.content.strip())
    drugs_yes_no_responses.append(yes_no_response.choices[0].message.content.strip())
    no_less_most_prompts.append(no_less_most_prompt)
    yes_no_prompts.append(yes_no_prompt)
    set_list.append(row["set_id"])
    fda_label_list.append(row["urls"])
    if index % 100 == 0:
      temp_df = pd.DataFrame({"generic_name":drugs_list,"less_determination":drugs_no_less_most_responses,"binary_determination":drugs_yes_no_responses,"no_less_most_prompts":no_less_most_prompts,"yes_no_prompts":yes_no_prompts,"set_id":set_list,"urls":fda_label_list})
      temp_df.to_csv("/content/drive/MyDrive/ClinicalTrials/Sunday_DICT_no_CoT_temp_"+str(index)+".csv")
final_df = pd.DataFrame({"generic_name":drugs_list,"less_determination":drugs_no_less_most_responses,"binary_determination":drugs_yes_no_responses,"no_less_most_prompts":no_less_most_prompts,"yes_no_prompts":yes_no_prompts,"set_id":set_list,"urls":fda_label_list})
final_df.to_csv("/content/drive/MyDrive/ClinicalTrials/Sunday_DICT_no_CoT_final.csv")

In [ ]:
dictrank_df

In [ ]:
final_df

In [ ]:
new_dictrank_df = pd.concat([dictrank_df,final_df],axis=1)

In [ ]:
new_dictrank_df["Answer"] = new_dictrank_df["DICT _ Concern"].map({"no":0,"less":1,"Less":1,"less ":1,"most":1,"ambiguous":1})
new_dictrank_df["Pred"] = new_dictrank_df["binary_determination"].map({"No.":0,"Yes.":1,"No":0,"Yes":1,"Less":1,"Most":1})

In [ ]:
print(new_dictrank_df[new_dictrank_df["Answer"].isna()])
print(new_dictrank_df[new_dictrank_df["Pred"].isna()])
print((new_dictrank_df["Answer"]==new_dictrank_df["Pred"]).sum()/len(new_dictrank_df))
new_dictrank_df = new_dictrank_df[new_dictrank_df["less_determination"].isin(["No","No.","Most","Most."])]
print((new_dictrank_df["Answer"]==new_dictrank_df["Pred"]).sum()/len(new_dictrank_df))

In [ ]:
new_dictrank_df = pd.concat([dictrank_df,final_df],axis=1)
new_dictrank_df["Answer"] = new_dictrank_df["DICT _ Concern"].map({"no":0,"less":1,"Less":1,"less ":1,"most":1,"ambiguous":1})
new_dictrank_df["Pred"] = new_dictrank_df["less_determination"].map({"No.":0,"Yes.":1,"No":0,"Yes":1,"Less":1,"Most":1})

print(new_dictrank_df[new_dictrank_df["Answer"].isna()])
print(new_dictrank_df[new_dictrank_df["Pred"].isna()])
print((new_dictrank_df["Answer"]==new_dictrank_df["Pred"]).sum()/len(new_dictrank_df))
new_dictrank_df = new_dictrank_df[new_dictrank_df["DICT _ Concern"].isin(["no","most"])]
print((new_dictrank_df["Answer"]==new_dictrank_df["Pred"]).sum()/len(new_dictrank_df))